### Capstone Project: Reducing Crime in San Francisco, Part 2

by Elton Yeo, DSI13

#### Contents:
- [Modelling](#Modelling)
- [Conclusion and Recommendations](#Conclusion-and-Recommendations)

In [79]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geopandas
from shapely.geometry import Point
from pygeocoder import Geocoder

from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import cross_val_score, KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from sklearn import metrics

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

%matplotlib inline

In [52]:
train=pd.read_csv('../data/train.csv')
test=pd.read_csv('../data/test.csv')

In [53]:
train.head()

,incident_day_of_week,incident_time,zip,preventable_crime,incident_hour
0,Wednesday,03:50,94103,1,3
1,Friday,09:30,94121,1,9
2,Friday,07:30,94103,1,7
3,Sunday,19:47,94118,1,19
4,Sunday,09:20,94107,1,9


In [54]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71444 entries, 0 to 71443
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   incident_day_of_week  71444 non-null  object
 1   incident_time         71444 non-null  object
 2   zip                   71444 non-null  int64 
 3   preventable_crime     71444 non-null  int64 
 4   incident_hour         71444 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 2.7+ MB


In [55]:
train.zip.astype('int64')

0        94103
1        94121
2        94103
3        94118
4        94107
         ...  
71439    94133
71440    94124
71441    94124
71442    94122
71443    94123
Name: zip, Length: 71444, dtype: int64

In [56]:
train=train.groupby(['incident_day_of_week', 'zip', 'incident_hour']).sum().reset_index()

In [57]:
train.sort_values('preventable_crime', ascending=False).head()

,incident_day_of_week,zip,incident_hour,preventable_crime
2480,Thursday,94103,19,114
44,Friday,94103,20,108
1274,Saturday,94103,23,107
43,Friday,94103,19,107
3701,Wednesday,94103,19,93


In [58]:
test=test.groupby(['incident_day_of_week', 'zip', 'incident_hour']).sum().reset_index()

In [59]:
test.sort_values('preventable_crime', ascending=False).head()

,incident_day_of_week,zip,incident_hour,preventable_crime
3078,Tuesday,94103,18,92
1967,Sunday,94109,0,89
42,Friday,94103,18,88
12,Friday,94102,12,87
3669,Wednesday,94102,18,86


In [60]:
#getting dummies from categorical data so that they can be processed by models
train = pd.get_dummies(train, columns = ['incident_day_of_week', 'zip', 'incident_hour'], drop_first=True)

#confirming that dummies have been created by checking the number of columns
train.shape

(4268, 56)

In [61]:
#getting dummies from categorical data so that they can be processed by models
test = pd.get_dummies(test, columns = ['incident_day_of_week', 'zip', 'incident_hour'], drop_first=True)

#confirming that dummies have been created by checking the number of columns
test.shape

(4262, 56)

#### Creating our train/test split and scaling

In [62]:
#create our features matrix X and target vector y
X = train.drop(['preventable_crime'], axis=1)
y = train['preventable_crime']

#train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

#scaling
ss = StandardScaler()
ss.fit(X_train)
X_train_sc = ss.transform(X_train)
X_test_sc = ss.transform(X_test) 

#### Baseline MSE score

In [97]:
base_df= pd.DataFrame(y_test)
base_df['baseline']= np.mean(y_test)
base_df

,preventable_crime,baseline
1702,1,16.863168
1173,23,16.863168
308,12,16.863168
1322,13,16.863168
2570,21,16.863168
...,...,...
2698,10,16.863168
1672,32,16.863168
4075,9,16.863168
604,3,16.863168


In [94]:
mean_squared_error(y_true=base_df.preventable_crime, y_pred=base_df.baseline)

247.07780927176287

#### Linear Regression

In [15]:
#instantiate our model
lr = LinearRegression()

#finding r-squared scores for linear regression
lr_score = cross_val_score(lr, X_train_sc, y_train, cv=10)
lr_score.mean()

0.7630160928049994

lr.fit(X_train_sc, y_train)
lr.score(X_test_sc, y_test)

#### Lasso

In [16]:
#instantiate our model and find optimal alpha
lasso = LassoCV(n_alphas=500)

#fitting to lasso
lasso.fit(X_train_sc, y_train)

#input optimal alpha
lasso_opt = Lasso(lasso.alpha_)

#finding r-squared scores for lasso
lasso_score = cross_val_score(lasso_opt, X_train_sc, y_train)
lasso_score.mean()

C:\Users\elton\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\elton\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


0.7647805078966549

#### Ridge

In [17]:
#instantiate our model and find optimal alpha
ridge_alphas=np.logspace(0, 5, 200)
ridge = RidgeCV(alphas=ridge_alphas)

#fitting to ridge
ridge.fit(X_train_sc, y_train)

#input optimal alpha
ridge_opt= Ridge(alpha=ridge.alpha_)

#finding r-squared scores for ridge
ridge_score = cross_val_score(ridge_opt, X_train_sc, y_train)
ridge_score.mean()

C:\Users\elton\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


0.7647855711923855

#### Random Forest with pipeline

In [18]:
#instantiate our model
rf = RandomForestRegressor()

#finding r-squared scores
rf_score = cross_val_score(rf, X_train_sc, y_train, cv=10)
rf_score.mean()

C:\Users\elton\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\elton\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\elton\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\elton\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\elton\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: F

0.7376869768802874

In [86]:
rf_parameters={
        'max_depth': np.arange(1, 5),
        'n_estimators': np.arange(1, 150),
        'min_samples_split': np.arange(2, 21),
        'min_samples_leaf': np.arange(1, 8)
}

rf=RandomizedSearchCV(RandomForestRegressor(), 
                      rf_parameters,
                      verbose=1)

rf.fit(X_train_sc, y_train)

C:\Users\elton\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    4.2s finished


RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators='warn',
                                                   n_jobs=None, oob_score=False,
                                                   rando...


In [87]:
rf.best_score_

0.41010535653638996

In [88]:
best_rf=rf.best_estimator_
best_rf.score(X_test_sc, y_test)

0.43095322229791355

#### XGBoost with pipeline

In [19]:
#instantiate our model
xgb = XGBRegressor()

#finding r-squared scores
xgb_score = cross_val_score(xgb, X_train_sc, y_train, cv=10)
xgb_score.mean()

0.8035506630853119

In [20]:
xgb.fit(X_train_sc, y_train)
xgb.score(X_test_sc, y_test)

0.8260213408536744

In [89]:
xgb_parameters={
        'max_depth': np.arange(1, 25),
        'n_estimators': np.arange(1, 1000), 
        'learning_rate': np.linspace(0.1, 1)
}

xgb=RandomizedSearchCV(XGBRegressor(), 
                      xgb_parameters,
                      verbose=1)

xgb.fit(X_train_sc, y_train)

C:\Users\elton\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  1.8min finished


RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, gamma=None,
                                          gpu_id=None, importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_weight=None, missing=nan,
                                          monoton...
       937, 938, 939, 940, 941, 942, 943, 944, 945, 946, 947, 948, 949,
       950, 951, 952, 953, 954, 955, 956, 957, 958, 959, 960, 961, 962,
       963, 964, 965, 966, 967, 968, 969, 970, 971, 972, 973, 974, 975,
       976, 977, 

In [90]:
xgb.best_score_

0.8033047760496923

In [91]:
best_xgb=xgb.best_estimator_
best_xgb.score(X_test_sc, y_test)

0.8190116066488419

In [100]:
y_pred=best_xgb.predict(X_test_sc)

mean_squared_error(y_test, y_pred)

44.71821573282025

#### Final model predicton and evaluation

In [21]:
X_valid = test.drop(['preventable_crime'], axis=1)
y_valid = test['preventable_crime']

#scaling my test set according to X_train
X_valid_sc = ss.transform(X_valid)

In [101]:
best_xgb.score(X_valid_sc, y_valid)

0.7899089050467243